# `o()`: attr-access for `dict`s
Wrap `dict`s (or kwarg lists) to allow direct `.`-accessing of items.

Before:
```python
x={'a':1,'b':2}  # given some dict
x['a'], x['b']   # access fields via "getitem" syntax
```
After:
```python
x=o({'a':1,'b':2})  # wrap dict in o()
x.a, x.b            # access members via "getattr" syntax
```
Or, instantiate directly with kwargs:
```python
x=o(a=1,b=2)
x.a, x.b
```

**Contents:**
- [Implementation](#Implementation)
- [Examples](#Examples)

## Implementation

In [1]:
class o(object):

    def __init__(self, *args, **kwargs):
        if len(args) > 1:
            raise ValueError(f'≤1 positional args required, got {len(args)}')
        
        if args:
            (data,) = args
            if type(data) is not dict:
                raise ValueError(f'Single-arg o() ctor call needs dict arg, not {type(data)}: {data}')
            if kwargs:
                raise ValueError(f'Positional dict arg is exclusive with kwargs: {data}, {kwargs}')
        else:
            data = kwargs
        
        K = '_data'
        if K in data:
            raise ValueError(f"Reserved key '{K}' found in 'data' dict: {data}")
            
        for k, v in data.items():
            super(o, self).__setattr__(k, v)
                                               
        super(o, self).__setattr__(K, data)

    def __dict__(self): return self._data
    
    def __setattr__(self, k, v):
        self._data[k] = v

    def __getattr__(self, k):
        try:
            return self._data[k]
        except KeyError:
            raise AttributeError(f'Key {k}')
    
    def __getitem__(self, k): return self._data[k]
    def __setitem__(self, k, v): self._data[k] = v
    
    def __str__(self): return str(self._data)
    def __repr__(self): return repr(self._data)

## Examples

In [2]:
o1 = o(a=1,b=2)

In [3]:
o1.a, o1.b

(1, 2)

In [4]:
import ur
from context import catch
with catch(AttributeError): o1.c

__enter__: ()
__exit__: exc_type=<class 'AttributeError'>, exc_value=AttributeError('Key c') (False)


In [5]:
x={'c':3,'d':4}
o2 = o(x)
o2.c, o2.d

(3, 4)

In [6]:
x['e']=5
o2.e

5

In [7]:
o2.c = 'ccc'
x

{'c': 'ccc', 'd': 4, 'e': 5}

In [8]:
o2

{'c': 'ccc', 'd': 4, 'e': 5}

In [9]:
str(o2)

"{'c': 'ccc', 'd': 4, 'e': 5}"